In [1]:
import pandas as pd

In [2]:
sample= pd.read_csv('labeled.csv')
sample

,id,keyword,location,text,target
0,3738,destroyed,USA,Black Eye 9: A space battle occurred at Star O...,0
1,853,bioterror,NaN,#world FedEx no longer to transport bioterror ...,0
2,10540,windstorm,"Palm Beach County, FL",Reality Training: Train falls off elevated tra...,1
3,5988,hazardous,USA,#Taiwan Grace: expect that large rocks trees m...,1
4,6328,hostage,Australia,New ISIS Video: ISIS Threatens to Behead Croat...,1
...,...,...,...,...,...
6466,4377,earthquake,ARGENTINA,#Earthquake #Sismo M 1.9 - 15km E of Anchorage...,1
6467,3408,derail,NaN,@EmiiliexIrwin Totally agree.She is 23 and kno...,0
6468,9794,trapped,NaN,Hollywood Movie About Trapped Miners Released ...,1
6469,10344,weapons,Beirut/Toronto,Friendly reminder that the only country to eve...,1


In [3]:
sample = sample.drop_duplicates('text', keep='last')
sample['target'].value_counts()

0    3674
1    2713
Name: target, dtype: int64

In [4]:
import re

def clean_up(s):
    
    lower = s.lower()
    no_url = re.sub('http\S+', '', lower)
    str_list = re.findall('[a-z]+', no_url)          
    
    return ' '.join(str_list)

In [5]:
import nltk
nltk.download('punkt')

def tokenize(s):
    return nltk.word_tokenize(s)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/rafaelsilva/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import*
nltk.download('wordnet')

def stem_and_lemmatize(l):
    
    lem_stem_list = [] 
    
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()
    
    for word in l:
        lem = lemmatizer.lemmatize(word)
        stem = stemmer.stem(lem)
        lem_stem_list.append(stem)
    
    return lem_stem_list

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/rafaelsilva/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def remove_stopwords(l):    
    
    filtered_sentence = [w for w in l if not w in stop_words]
    
    return filtered_sentence

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rafaelsilva/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
def all_in_one(new_input):
    
    new_input=clean_up(new_input)
    new_input=tokenize(new_input)
    new_input=stem_and_lemmatize(new_input)
    new_input=remove_stopwords(new_input)
    
    return new_input

In [9]:
sample['processed']=sample['text'].apply(all_in_one)

/var/folders/mv/pzkkwg_132x_kq3wgrqvf2jr0000gn/T/ipykernel_4846/4038282554.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['processed']=sample['text'].apply(all_in_one)


In [10]:
sample.tail()

,id,keyword,location,text,target,processed
6466,4377,earthquake,ARGENTINA,#Earthquake #Sismo M 1.9 - 15km E of Anchorage...,1,"[earthquak, sismo, km, e, anchorag, alaska, ti..."
6467,3408,derail,NaN,@EmiiliexIrwin Totally agree.She is 23 and kno...,0,"[emiiliexirwin, total, agre, know, birth, cont..."
6468,9794,trapped,NaN,Hollywood Movie About Trapped Miners Released ...,1,"[hollywood, movi, trap, miner, releas, chile, ..."
6469,10344,weapons,Beirut/Toronto,Friendly reminder that the only country to eve...,1,"[friendli, remind, onli, countri, ever, use, n..."
6470,1779,buildings%20on%20fire,NaN,Buildings are on fire and they have time for a...,1,"[build, fire, time, busi, meet, thestrain]"


In [11]:
new_list = []

for tweet in sample['processed']:
    for word in tweet:
        new_list.append(word)
        
new_list=nltk.FreqDist(new_list)

In [12]:
def find_features(document):
    
    words = set(document)
    features = {}
    
    for w in new_list:
        features[w] = (w in words)

    return features

In [13]:
featuresets = [(find_features(tweet), target) for (tweet, target) in list(zip(sample['processed'], sample['target']))]

print(len(featuresets))

6387


In [14]:
training_set = featuresets[:round(len(featuresets)/2)]

testing_set = featuresets[round(len(featuresets)/2):]

classifier = nltk.NaiveBayesClassifier.train(training_set)

In [15]:
nltk.classify.accuracy(classifier, testing_set)

0.7813968055120576